In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [2]:
year = 2023
loads = pd.read_csv(f'../data/Prepared data/capitl_{year}.csv')
weather = pd.read_csv(f'../data/Prepared data/{year}_weather_data.csv')

In [3]:
weather

,valid_time_gmt,temp,wspd,pressure,precip_hrly
0,2023-01-01 00:19,53,3.0,29.69,0.00
1,2023-01-01 00:51,53,7.0,29.72,0.00
2,2023-01-01 01:24,54,3.0,29.70,0.02
3,2023-01-01 01:51,54,12.0,29.71,0.02
4,2023-01-01 02:51,54,12.0,29.74,0.00
...,...,...,...,...,...
10183,2023-12-31 19:51,43,8.0,29.98,0.00
10184,2023-12-31 20:51,43,8.0,29.98,0.00
10185,2023-12-31 21:51,43,8.0,29.98,0.00
10186,2023-12-31 22:51,43,8.0,29.97,0.00


In [4]:
loads

,timestamp,name,ptid,load
0,01/01/2023 00:00:00,CAPITL,61757,1153.4263
1,01/01/2023 00:05:00,CAPITL,61757,1138.7476
2,01/01/2023 00:10:00,CAPITL,61757,1134.5775
3,01/01/2023 00:15:00,CAPITL,61757,1145.0581
4,01/01/2023 00:20:00,CAPITL,61757,1137.9814
...,...,...,...,...
107270,12/31/2023 23:35:00,CAPITL,61757,1197.5303
107271,12/31/2023 23:40:00,CAPITL,61757,1209.7126
107272,12/31/2023 23:45:00,CAPITL,61757,1192.3306
107273,12/31/2023 23:50:00,CAPITL,61757,1196.0747


In [5]:
weather = weather.rename(columns={"valid_time_gmt": "timestamp"})

In [6]:
# weather['date'] = weather.date.apply(lambda x: pd.to_datetime(x).date())
# weather['timeest'] = weather.Time.apply(lambda x: pd.to_datetime(x).time())
# foo = weather[['date', 'timeest']].astype(str)
# weather['timestamp'] = pd.to_datetime(foo['date'] + ' ' + foo['timeest'])
print(weather.columns)
weather['timestamp'] = weather.timestamp.apply(lambda x: pd.to_datetime(x))
loads['timestamp'] = loads.timestamp.apply(lambda x: pd.to_datetime(x))



Index(['timestamp', 'temp', 'wspd', 'pressure', 'precip_hrly'], dtype='object')


In [7]:
weather

,timestamp,temp,wspd,pressure,precip_hrly
0,2023-01-01 00:19:00,53,3.0,29.69,0.00
1,2023-01-01 00:51:00,53,7.0,29.72,0.00
2,2023-01-01 01:24:00,54,3.0,29.70,0.02
3,2023-01-01 01:51:00,54,12.0,29.71,0.02
4,2023-01-01 02:51:00,54,12.0,29.74,0.00
...,...,...,...,...,...
10183,2023-12-31 19:51:00,43,8.0,29.98,0.00
10184,2023-12-31 20:51:00,43,8.0,29.98,0.00
10185,2023-12-31 21:51:00,43,8.0,29.98,0.00
10186,2023-12-31 22:51:00,43,8.0,29.97,0.00


In [8]:
from sklearn.neighbors import NearestNeighbors

def find_nearest(group, match, groupname):
    nbrs = NearestNeighbors(n_neighbors=1).fit(match['timestamp'].values[:, None])
    dist, ind = nbrs.kneighbors(group['timestamp'].values[:, None])

    group['nearesttime'] = match['timestamp'].values[ind.ravel()]
    return group

In [9]:
loads = find_nearest(loads,weather,'timestamp')

In [10]:
loads

,timestamp,name,ptid,load,nearesttime
0,2023-01-01 00:00:00,CAPITL,61757,1153.4263,2023-01-01 00:19:00
1,2023-01-01 00:05:00,CAPITL,61757,1138.7476,2023-01-01 00:19:00
2,2023-01-01 00:10:00,CAPITL,61757,1134.5775,2023-01-01 00:19:00
3,2023-01-01 00:15:00,CAPITL,61757,1145.0581,2023-01-01 00:19:00
4,2023-01-01 00:20:00,CAPITL,61757,1137.9814,2023-01-01 00:19:00
...,...,...,...,...,...
107270,2023-12-31 23:35:00,CAPITL,61757,1197.5303,2023-12-31 23:51:00
107271,2023-12-31 23:40:00,CAPITL,61757,1209.7126,2023-12-31 23:51:00
107272,2023-12-31 23:45:00,CAPITL,61757,1192.3306,2023-12-31 23:51:00
107273,2023-12-31 23:50:00,CAPITL,61757,1196.0747,2023-12-31 23:51:00


In [11]:
full = loads.merge(weather, left_on='nearesttime', right_on='timestamp')
print(full.columns)
print(full)

full = full[['timestamp_x', 'load', 'nearesttime', 'temp', 'wspd', 'pressure', 'precip_hrly']].rename(columns={'timestamp_x': 'timestamp', 'nearesttime':'weathertime'})

Index(['timestamp_x', 'name', 'ptid', 'load', 'nearesttime', 'timestamp_y',
       'temp', 'wspd', 'pressure', 'precip_hrly'],
      dtype='object')
               timestamp_x    name   ptid       load         nearesttime  \
0      2023-01-01 00:00:00  CAPITL  61757  1153.4263 2023-01-01 00:19:00   
1      2023-01-01 00:05:00  CAPITL  61757  1138.7476 2023-01-01 00:19:00   
2      2023-01-01 00:10:00  CAPITL  61757  1134.5775 2023-01-01 00:19:00   
3      2023-01-01 00:15:00  CAPITL  61757  1145.0581 2023-01-01 00:19:00   
4      2023-01-01 00:20:00  CAPITL  61757  1137.9814 2023-01-01 00:19:00   
...                    ...     ...    ...        ...                 ...   
107270 2023-12-31 23:35:00  CAPITL  61757  1197.5303 2023-12-31 23:51:00   
107271 2023-12-31 23:40:00  CAPITL  61757  1209.7126 2023-12-31 23:51:00   
107272 2023-12-31 23:45:00  CAPITL  61757  1192.3306 2023-12-31 23:51:00   
107273 2023-12-31 23:50:00  CAPITL  61757  1196.0747 2023-12-31 23:51:00   
107274 2023-12-

In [12]:
pday = pd.Timedelta('1 day')

def get_prev_days(x, n_days):
    try:
        lo = full[full.timestamp == x - n_days*pday].load.values[0]
    except:
        lo = full[full.timestamp == x].load.values[0]
    return lo 

In [13]:
full['dow'] = full.timestamp.apply(lambda x: x.dayofweek)
full['doy'] = full.timestamp.apply(lambda x: x.dayofyear)
full['day'] = full.timestamp.apply(lambda x: x.day)
full['month'] = full.timestamp.apply(lambda x: x.month)
full['hour'] = full.timestamp.apply(lambda x: x.hour)
full['minute'] = full.timestamp.apply(lambda x: x.hour*60 + x.minute)
full['year'] = full.timestamp.apply(lambda x: x.year)

# full['t_m24'] = full.timestamp.apply(get_prev_days, args=(1,))
# full['t_m48'] = full.timestamp.apply(get_prev_days, args=(2,))
# full['tdif'] = full['load'] - full['t_m24']

In [14]:
# full['Temperature'] = full['Temperature'].map(lambda x: x[:-2])
# full['Dew Point'] = full['Dew Point'].map(lambda x: x[:-2])
# full['Humidity'] = full['Humidity'].map(lambda x: x[:-2])
# full['Pressure'] = full['Pressure'].map(lambda x: x[:-3])
# full['Wind Speed'] = full['Wind Speed'].map(lambda x: x[:-4])
# full['precipitation'] = full['precipitation'].map(lambda x: x[:-3])

In [15]:
full

,timestamp,load,weathertime,temp,wspd,pressure,precip_hrly,dow,doy,day,month,hour,minute,year
0,2023-01-01 00:00:00,1153.4263,2023-01-01 00:19:00,53,3.0,29.69,0.0,6,1,1,1,0,0,2023
1,2023-01-01 00:05:00,1138.7476,2023-01-01 00:19:00,53,3.0,29.69,0.0,6,1,1,1,0,5,2023
2,2023-01-01 00:10:00,1134.5775,2023-01-01 00:19:00,53,3.0,29.69,0.0,6,1,1,1,0,10,2023
3,2023-01-01 00:15:00,1145.0581,2023-01-01 00:19:00,53,3.0,29.69,0.0,6,1,1,1,0,15,2023
4,2023-01-01 00:20:00,1137.9814,2023-01-01 00:19:00,53,3.0,29.69,0.0,6,1,1,1,0,20,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107270,2023-12-31 23:35:00,1197.5303,2023-12-31 23:51:00,44,9.0,29.98,0.0,6,365,31,12,23,1415,2023
107271,2023-12-31 23:40:00,1209.7126,2023-12-31 23:51:00,44,9.0,29.98,0.0,6,365,31,12,23,1420,2023
107272,2023-12-31 23:45:00,1192.3306,2023-12-31 23:51:00,44,9.0,29.98,0.0,6,365,31,12,23,1425,2023
107273,2023-12-31 23:50:00,1196.0747,2023-12-31 23:51:00,44,9.0,29.98,0.0,6,365,31,12,23,1430,2023


In [16]:
full.to_csv(f'{year}_features.csv',encoding='utf-8-sig', index=False)